In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

In [4]:
dataset = load_dataset("allocine")

In [5]:
# Charger les mots positifs depuis le fichier
with open("positive_word.txt", "r") as f:
    positive_words = f.read().splitlines()

# Charger les mots négatifs depuis le fichier
with open("negative_word.txt", "r") as f:
    negative_words = f.read().splitlines()

In [7]:

# Diviser le dataset en données d'entraînement, de validation et de test
train_texts = [example["review"] for example in dataset["train"]]
train_labels = [example["label"] for example in dataset["train"]]
test_texts = [example["review"] for example in dataset["test"]]
test_labels = [example["label"] for example in dataset["test"]]


In [8]:
# Vectoriser les données textuelles (utilisation de TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [9]:
# Tokenization et lemmatisation avec NLTK
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('french'))

In [10]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token.lower() not in stop_words]
    return " ".join(tokens)

In [11]:
train_texts_preprocessed = [preprocess_text(text) for text in train_texts]
test_texts_preprocessed = [preprocess_text(text) for text in test_texts]

# Vectoriser les données textuelles avec TF-IDF en utilisant scikit-learn
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts_preprocessed)
X_test = vectorizer.transform(test_texts_preprocessed)

# Entraîner un modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, train_labels)

LogisticRegression()

In [42]:
# Enregistrer le modèle et le vectorizer en tant que fichiers pickle
with open('model_vectorizer.pkl', 'wb') as file:
    pickle.dump((model, vectorizer), file)

# Charger le modèle et le vectorizer depuis le fichier pickle
with open('model_vectorizer.pkl', 'rb') as file:
    loaded_model, loaded_vectorizer = pickle.load(file)

In [51]:
# Faire des prédictions sur les commentaires de test
predictions = loaded_model.predict(X_test)

# Calculer et afficher l'exactitude du modèle sur les données de test
accuracy = (predictions == test_labels).mean()
print(f"Exactitude du modèle sur les données de test : {accuracy}")

Exactitude du modèle sur les données de test : 0.91775


In [ ]:
# Saisir des commentaires en entrée et effectuer des prédictions
while True:
    input_comment = input("Saisissez un commentaire (ou 'exit' pour quitter) : ")
    if input_comment.lower() == 'exit':
        break
    
    input_comment_preprocessed = preprocess_text(input_comment)
    input_comment_vectorized = loaded_vectorizer.transform([input_comment_preprocessed])
    prediction = loaded_model.predict(input_comment_vectorized)[0]
    
    sentiment = "Positif" if prediction == 1 else "Négatif"
    print(f"Sentiment prédit : {sentiment}")

Saisissez un commentaire (ou 'exit' pour quitter) : bon film
Sentiment prédit : Positif
Saisissez un commentaire (ou 'exit' pour quitter) : nul , c'est trop
Sentiment prédit : Négatif
Saisissez un commentaire (ou 'exit' pour quitter) : c'est trop cool 
Sentiment prédit : Négatif
